In [32]:
import os
from dataclasses import dataclass
from pathlib import Path

from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

import urllib.request as request
import zipfile
from cnn_classifier import logger
from cnn_classifier.utils.common import get_size

from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [14]:
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Chest-X-Ray-Pathology-Classification'

In [15]:
os.chdir("../")

In [16]:
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Chest-X-Ray-Pathology-Classification'

In [33]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [37]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-04 16:35:39,452: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-04 16:35:39,455: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-04 16:35:39,456: INFO: common] created directory at: artifacts
[2023-10-04 16:35:39,458: INFO: common] created directory at: artifacts/data_ingestion
[2023-10-04 16:35:39,458: INFO: 1776151970] File already exists of size: ~ 256183 KB
